In [1]:
import hoomd
import numpy as np
from numba import njit
from hoomd import nlist_plugin


In [ ]:
import copy


@njit
def _neb_force_compute(segments, k, rtags, neb_force, force, pos, N, R1, R2, tau1, tau2, tau):

    for i in range(1, len(segments)-1):
        head = segments[i]
        head_n1 = segments[i-1]
        head_p1 = segments[i+1]

        for j in range(N):
            R1[j] = pos[rtags[head+j]] - pos[rtags[head_n1+j]]
            R2[j] = pos[rtags[head_p1+j]] - pos[rtags[head+j]]
        norm1 = np.linalg.norm(R1)
        norm2 = np.linalg.norm(R2)
        for j in range(N):
            tau1[j] = R1[j]/norm1
            tau2[j] = R2[j]/norm2
        tau = tau1 + tau2
        tau /= np.linalg.norm(tau)

        nudge_force = k * np.sum((R2 - R1)* tau) - np.sum(force * tau)
        for j in range(N):
            neb_force[rtags[head+j]] = nudge_force*tau[j]


class NudgedForce(hoomd.md.force.Custom):
    def __init__(self, k: float, segments: np.ndarray, force: hoomd.md.force.Force):
        self._k = k
        self._segments = segments
        self._force = force
        self._N = segments[1] - segments[0]
        self._tau1 = np.zeros(self._N, 3)
        self._tau2 = np.zeros(self._N, 3)
        self._tau = np.zeros(self._N, 3)
        self._R1 = np.zeros(self._N, 3)
        self._R2 = np.zeros(self._N, 3)
        super().__init__(aniso=False)

    def set_forces(self, timestep):
        force = self._force.forces  # doesn't need rtags
        with self._state.cpu_local_snapshot as snap:
            with self.cpu_local_force_arrays as arrays:
                _neb_force_compute(self._segments, self._k, snap.particles.rtag,
                                   arrays.force, force, snap.particles.position,
                                   self._N, self._R1, self._R2, self._tau1, self._tau2, self._tau)


class NudgedElasticBand:

    def __init__(self, sim: hoomd.Simulation, deepcopy=False):
        if deepcopy:
            self._sim = copy.deepcopy(sim)
        else:
            self._sim = sim
        self._setup = False

    def setup(self, initial_snap=None, final_snap=None, n_images=10):

        # get a clean simulation object
        sim = self._sim

        integrator = sim.operations.integrator
        assert isinstance(integrator, hoomd.md.Integrator)
        orig_force = integrator.forces[0]
        orig_method = integrator.methods[0]

        if initial_snap is None:
            initial_snap = sim.state.get_snapshot()

        if final_snap is None:
            raise ValueError("final_state must be specified")

        N = initial_snap.particles.N
        types = initial_snap.particles.types
        typeid = initial_snap.particles.typeid
        if N != final_snap.particles.N:
            raise ValueError(
                "initial and final states must have the same number of particles")
        if types != final_snap.particles.types:
            raise ValueError(
                "initial and final states must have the same particle types")
        if typeid != final_snap.particles.typeid:
            raise ValueError(
                "initial and final states must have the same particle types")

        pos = np.zeros((n_images + 2, N, 3))
        pos[0] = initial_snap.particles.position
        pos[-1] = final_snap.particles.position
        forces = []
        segments = [0]
        head = N
        for i in range(1, n_images + 1):
            nlist = hoomd.md.nlist.Cell()
            
            segments.append(head)
            pos[i] = (n_images - i) / n_images * \
                pos[0] + i / n_images * pos[-1]
            head += N
        segments.append(head)
        segments = np.array(segments)

        # create a new snapshot
        snap = hoomd.Snapshot()
        total_images = n_images + 2
        new_N = N * total_images
        snap.particles.N = new_N
        snap.particles.position = pos.reshape(new_N, 3)
        snap.particles.types = types
        snap.particles.typeid = np.repeat(typeid, total_images)

        sim.state.set_snapshot(snap)

    def run(self, steps):
        self._sim.run(steps)
